# 6.Run object detection using Detectron2

This notebook was used to identify the objects in each photograph in the De Boer Scene Detection dataset. The object detection is adapted from the [Detectron2 Tutorial Colab notebook](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5).

Detectron2 is avaialble here: https://github.com/facebookresearch/detectron2/tree/main

## Install detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44090 sha256=a2825b8a31b6366a716fda30d80ae6a225bb49175ea3790e40ff7570a6fb6e42
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.6.9 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatibl

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## Run a pretrained detectron2 on De Boer images

In [ ]:
import pickle
import json

# helpers to load and save pickle and json files

def loadPKL(path):
  infile = open(path,'rb')
  X = pickle.load(infile)
  print('Loaded ' + path.split('/')[-1])
  return X

def savePKL(data, path):
  with open(path, 'wb') as f:
    pickle.dump(data, f)
  print('Saved ' + path.split('/')[-1])

def loadJSON(path):
  infile = open(path,'rb')
  X = json.load(infile)
  print('Loaded ' + path.split('/')[-1])
  return X

def saveJSON(data, path):
  with open(path, 'w') as jsonfile:
    json.dump(data, jsonfile)
  print('Saved ' + path.split('/')[-1])

Load De Boer images and metadata

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
root_dir = '/content/drive/MyDrive/dating-images'
data_folder = root_dir + '/data/'
model_folder = root_dir + '/models/'
image_folder = root_dir + '/data/images/'
out_folder = root_dir + '/output/'

In [ ]:
image_data = loadJSON(data_folder + 'image_data.json')

Loaded image_data.json


In [ ]:
print(image_data[:5])
print(len(image_data))

[['../scene_detection/images/soccer/NL-HlmNHA_1478_20083K00_22.jpg', '19-11-1980', 1980, 'soccer', 'train'], ['../scene_detection/images/portrait/NL-HlmNHA_1478_00528A00_05.jpg', '09-02-1951', 1951, 'portrait', 'train'], ['../scene_detection/images/crowd/NL-HlmNHA_1478_40275K00_05.jpg', '01-10-1994', 1994, 'crowd', 'train'], ['../scene_detection/images/portrait/NL-HlmNHA_1478_00783K00_18.jpg', '15-05-1963', 1963, 'portrait', 'train'], ['../scene_detection/images/baseball/NL-HlmNHA_1478_26496K00_36.jpg', '06-07-1986', 1986, 'baseball', 'train']]
39866


Create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

[06/19 07:50:27 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


model_final_f10217.pkl: 178MB [00:00, 234MB/s]                           


Run object detection on each image. Record the objects and their respective confidence scores.

In [ ]:
out_file_1 = 'objects_1.json'
out_file_2 = 'scores_1.json'
objects = []
scores = []
index = 0

for i, item in enumerate(image_data[index:]):
  path = image_folder + item[0].split('images/')[-1]
  image = cv2.imread(path)
  outputs = predictor(image)
  detected_objects = outputs["instances"].pred_classes.tolist()
  confidence_scores = outputs["instances"].scores.tolist()
  objects.append(detected_objects)
  scores.append(confidence_scores)
  if (i+1)%1000 == 0:
    saveJSON(objects, out_folder + out_file_1) # save a copy intermittently
    saveJSON(scores, out_folder + out_file_2)
    print('Saved at index ' + str(i))

saveJSON(objects, out_folder + out_file_1)
saveJSON(scores, out_folder + out_file_2)

## Store predictions, labels and detection results in one file

The class names corresponding to the output instances

In [ ]:
className = {0: u'__background__',
 1: u'person',
 2: u'bicycle',
 3: u'car',
 4: u'motorcycle',
 5: u'airplane',
 6: u'bus',
 7: u'train',
 8: u'truck',
 9: u'boat',
 10: u'traffic light',
 11: u'fire hydrant',
 12: u'stop sign',
 13: u'parking meter',
 14: u'bench',
 15: u'bird',
 16: u'cat',
 17: u'dog',
 18: u'horse',
 19: u'sheep',
 20: u'cow',
 21: u'elephant',
 22: u'bear',
 23: u'zebra',
 24: u'giraffe',
 25: u'backpack',
 26: u'umbrella',
 27: u'handbag',
 28: u'tie',
 29: u'suitcase',
 30: u'frisbee',
 31: u'skis',
 32: u'snowboard',
 33: u'sports ball',
 34: u'kite',
 35: u'baseball bat',
 36: u'baseball glove',
 37: u'skateboard',
 38: u'surfboard',
 39: u'tennis racket',
 40: u'bottle',
 41: u'wine glass',
 42: u'cup',
 43: u'fork',
 44: u'knife',
 45: u'spoon',
 46: u'bowl',
 47: u'banana',
 48: u'apple',
 49: u'sandwich',
 50: u'orange',
 51: u'broccoli',
 52: u'carrot',
 53: u'hot dog',
 54: u'pizza',
 55: u'donut',
 56: u'cake',
 57: u'chair',
 58: u'couch',
 59: u'potted plant',
 60: u'bed',
 61: u'dining table',
 62: u'toilet',
 63: u'tv',
 64: u'laptop',
 65: u'mouse',
 66: u'remote',
 67: u'keyboard',
 68: u'cell phone',
 69: u'microwave',
 70: u'oven',
 71: u'toaster',
 72: u'sink',
 73: u'refrigerator',
 74: u'book',
 75: u'clock',
 76: u'vase',
 77: u'scissors',
 78: u'teddy bear',
 79: u'hair drier',
 80: u'toothbrush'}

### Combine object predictions and scores

In [ ]:
objects_class_names = []
for line in objects:
  pred_class_names = list(map(lambda x: className[x+1], line))
  objects_class_names.append(pred_class_names)

In [ ]:
def merge(list1, list2):
    merged_list = list(zip(list1, list2))
    return merged_list

detection_results = []
for i, line in enumerate(objects_class_names):
  detection_results.append(merge(line, scores[i]))

print(objects[0])
print(scores[0])

print(detection_results[0])
print(len(detection_results))


[0, 0, 0]
[0.9997135996818542, 0.99442058801651, 0.9393302202224731]
[('person', 0.9997135996818542), ('person', 0.99442058801651), ('person', 0.9393302202224731)]
39866


### Load labels and models

In [ ]:
image_data = loadJSON(data_folder + 'image_data.json') # image_data = [[path, data, year, scene, train/test], ... ]

years = list(zip(*image_data))[2]
scenes = list(zip(*image_data))[3]

Loaded image_data.json


In [ ]:
embeddings = loadPKL(data_folder + 'embeddings.pkl')
embeddings_colorized = loadPKL(data_folder + 'embeddings_colorized.pkl')

classifier = loadPKL(model_folder + '20230621-1014_logistic_year.pkl')
classifier_colorized = loadPKL(model_folder + '20230621-1033_logistic_year_colorized.pkl')

years_pred = classifier.predict(embeddings)
years_colorized_pred = classifier_colorized.predict(embeddings_colorized)

Loaded embeddings.pkl
Loaded embeddings_colorized.pkl
Loaded 20230621-1014_logistic_year.pkl
Loaded 20230621-1033_logistic_year_colorized.pkl


Save a json object with all predicted years, actual years, scenes, train/test labels and object detection results


In [ ]:
output = []
for i, item in enumerate(image_data):
  out_dict = {
    "year_pred": int(years_pred[i]),
    "year_colorized_pred": int(years_colorized_pred[i]),
    "actual_year": years[i],
    "scene": scenes[i],
    "train_test": item[-1],
    "objects": detection_results[i]
  }
  output.append(out_dict)

print(len(output))
print(output[:5])

39866
[{'year_pred': 1980, 'year_colorized_pred': 1980, 'actual_year': 1980, 'scene': 'soccer', 'train_test': 'train', 'objects': [['person', 0.9997135996818542], ['person', 0.99442058801651], ['person', 0.9393302202224731]]}, {'year_pred': 1955, 'year_colorized_pred': 1960, 'actual_year': 1951, 'scene': 'portrait', 'train_test': 'train', 'objects': [['person', 0.9989057779312134]]}, {'year_pred': 1985, 'year_colorized_pred': 1973, 'actual_year': 1994, 'scene': 'crowd', 'train_test': 'train', 'objects': [['person', 0.9985594153404236], ['person', 0.9966326355934143], ['person', 0.9945083260536194], ['person', 0.9925635457038879], ['person', 0.9872242212295532], ['person', 0.9808698892593384], ['person', 0.9771793484687805], ['person', 0.9487467408180237], ['person', 0.9238803386688232], ['person', 0.9082883596420288], ['person', 0.8778145909309387], ['person', 0.876467764377594], ['person', 0.8670186400413513], ['person', 0.8551080226898193], ['person', 0.8428810238838196], ['person', 

In [ ]:
saveJSON(output, data_folder + 'year_predictions_new.json')

Saved year_predictions_new.json
